# Caso de Estudio: Cyclistic Bike-Share (Q1 2024)

**Autora:** Laura Agostina Spena  
**Fecha:** Julio 2025  
**Rol:** Analista de Datos Junior (Certificación Google Data Analytics)  

Este cuaderno documenta el análisis realizado como proyecto final del **Certificado de Google Data Analytics** (Coursera).  
El objetivo es explorar las diferencias de comportamiento entre **usuarios miembros** y **usuarios casuales** del sistema de bicicletas compartidas ficticio **Cyclistic**, con el fin de proponer estrategias de marketing que fomenten la conversión de usuarios casuales en miembros de suscripción.

---

## 📚 Índice del notebook

1. [Caso de Estudio: Cyclistic Bike-Share (Q1 2024)](#caso-de-estudio-cyclistic-bike-share-q1-2024)
   - [Contexto del caso](#contexto-del-caso)
   - [Herramientas utilizadas](#herramientas-utilizadas)

2. [Análisis preliminar con dataset incompleto](#descripción-del-dataset-versión-preliminar)
   - [Descripción del dataset (versión preliminar)](#descripción-del-dataset-versión-preliminar)
   - [Limpieza y preparación inicial de datos (tabla sesgada)](#limpieza-y-preparación-inicial-de-datos-tabla-sesgada)
   - [Análisis Exploratorio de Datos (EDA) - Versión preliminar](#análisis-exploratorio-de-datos-eda---versión-preliminar)

3. [Detección del sesgo y revision critica](#detección-del-sesgo-revisión-crítica-del-proceso-de-limpieza)

4. [Análisis con tabla corregida](#limpieza-de-datos-realizada-en-bigquery)
   - [Limpieza de datos realizada en BigQuery](#limpieza-de-datos-realizada-en-bigquery)
   - [Validación de la importación del dataset final](#validación-de-la-importación-del-dataset-final)
   - [Nuevo Análisis Exploratorio de Datos (EDA)](#nuevo-análisis-exploratorio-de-datos-eda)

5. [Comparación entre análisis](#revisión-comparativa-entre-análisis-preliminar-y-análisis-con-datos-íntegros)

6. [Conclusiones y próximos pasos](#hallazgos-clave-del-análisis-tabla-íntegra)
   - [Hallazgos clave del análisis (tabla íntegra)](#hallazgos-clave-del-análisis-tabla-íntegra)
   - [Recomendaciones y próximos pasos](#recomendaciones-y-próximos-pasos)
   - [Conclusión del proceso: del dato a la experiencia](#conclusión-del-proceso-del-dato-a-la-experiencia)

7. [Si solo vas a leer una sección, que sea esta](#si-solo-vas-a-leer-una-sección-que-sea-esta)

---

### Contexto del caso

Cyclistic es una empresa ficticia basada en un dataset real de **Divvy Bikes (Motivate International Inc.)**. Para este análisis, se utilizó la información correspondiente al **primer trimestre del año 2024 (enero, febrero y marzo)**.

Los datos pueden consultarse públicamente en el portal de [Divvy Tripdata](https://divvy-tripdata.s3.amazonaws.com/index.html).

---

### Herramientas utilizadas:

- **Google Sheets** → limpieza inicial de archivos individuales  
- **BigQuery (SQL)** → unificación, limpieza estructurada y análisis  
- **Google Sheets** → depuración de tablas para visualización  
- **Tableau** → visualización de datos en dashboard interactivo  
- **Canva** → presentación con hallazgos, próximos pasos y recomendaciones  
- **Kaggle (R)** → documentación reproducible del análisis final

---

> ⚠️ Este cuaderno refleja una segunda versión del análisis, basada en una nueva tabla unificada y validada. El primer análisis contenía una omisión parcial de datos en el mes de marzo, lo cual fue posteriormente detectado y corregido. Esta documentación detalla todo el proceso corregido paso a paso.


In [ ]:
# Librerías necesarias
library(tidyverse)
library(lubridate)
library(scales)
library(janitor)
library(ggplot2)
library(readr)

In [ ]:
# Cargar dataset original (versión preliminar - marzo incompleto)

# Este archivo CSV fue exportado desde BigQuery tras la limpieza inicial,
# basada en una versión incompleta del mes de marzo 2024.
# Se utilizó en la primera fase del análisis antes de detectar la omisión de datos.

# Ruta al archivo CSV
ruta_csv <- "/kaggle/input/cyclistic-casestudy/tripsdata_q1_2024.csv"

# Leer el archivo sin encabezados
df <- read_csv(ruta_csv, col_names = FALSE)

# Asignar nombres correctos de columna (definidos manualmente)
colnames(df) <- c(
  "ride_id",
  "rideable_type",
  "started_at",
  "ended_at",
  "start_station_name",
  "start_station_id",
  "end_station_name",
  "end_station_id",
  "start_lat",
  "start_lng",
  "end_lat",
  "end_lng",
  "member_casual",
  "ride_length",
  "day_of_week",
  "duration_min"
)

# Limpiar los nombres por si acaso 
df <- janitor::clean_names(df)

# Ver estructura para confirmar
glimpse(df)


---

## Descripción del dataset (versión preliminar)

Este dataset contiene **293.051 registros de viajes** correspondientes al primer trimestre del año 2024.  
Fue generado a partir de una tabla exportada desde BigQuery, en la que luego se detectó que el mes de marzo estaba **incompleto**.  
Este conjunto de datos se utilizó en la fase inicial del análisis.

### Variables principales incluidas:

- `rideable_type`: tipo de bicicleta utilizada  
- `member_casual`: tipo de usuario (miembro o casual)  
- `started_at` y `ended_at`: fecha y hora de inicio y fin del viaje  
- `start_station_name` y `end_station_name`: nombres de las estaciones  
- `start_lat` / `start_lng` y `end_lat` / `end_lng`: coordenadas geográficas  
- `ride_length`: duración del viaje (en formato de tiempo)  
- `day_of_week`: día de la semana del viaje  
- `ride_length_minutes`: duración del viaje en minutos

---

📌 **Objetivo del análisis**:  
Detectar patrones y diferencias de comportamiento entre los dos tipos de usuarios (**miembros** y **casuales**) que permitan formular recomendaciones para estrategias de conversion para el area de marketing.


---

## Limpieza y preparación inicial de datos (tabla sesgada)

Esta etapa de limpieza se realizó sobre el dataset exportado desde BigQuery y cargado en Kaggle, correspondiente a la versión preliminar (aún sin detectar el sesgo de marzo incompleto).  
El objetivo fue preparar la información para realizar un análisis exploratorio y generar visualizaciones efectivas del comportamiento de los usuarios.

### Pasos realizados:

- **Conversión de formatos de fecha y hora:**  
  Las columnas `started_at` y `ended_at` fueron convertidas al formato datetime estándar, para facilitar el análisis temporal.

- **Cálculo y validación de la duración del viaje:**  
  Se calculó una nueva columna con la duración del viaje en minutos (`ride_length_minutes`), basada en la diferencia entre las fechas de inicio y fin. Esta variable fue comparada con la duración original para validar consistencia.

- **Eliminación de registros anómalos:**  
  Se descartaron registros con duración:
  - Nula o negativa
  - Mayores a 180 minutos (3 horas), por considerarse outliers que podrían distorsionar el análisis

- **Creación de variables clave adicionales:**
  - `month`: mes del viaje
  - `day_of_week_text`: día de la semana abreviado (Mon, Tue, etc.)
  - `is_weekend`: indicador de fin de semana
  - `hour_of_day`: hora de inicio del viaje
  - `part_of_day`: franja horaria categorizada (madrugada, mañana, tarde, noche)

---

Estos pasos permitieron transformar el dataset preliminar en una base apta para el análisis exploratorio, desde la perspectiva del comportamiento de los usuarios **miembros** y **casuales**.



In [ ]:
# Manejo de fechas y horas en el dataset preliminar (sesgado)
library(lubridate)  

# Convertir formato datetime
df <- df %>%
  mutate(
    started_at = ymd_hms(started_at, tz = "UTC"),
    ended_at = ymd_hms(ended_at, tz = "UTC")
  )

# Recalcular duración en minutos (para validar consistencia)
df <- df %>%
  mutate(
    duration_min_calc = as.numeric(difftime(ended_at, started_at, units = "mins"))
  )

# Crear día de la semana como texto abreviado (Mon, Tue, etc.)
df <- df %>%
  mutate(
    day_of_week_text = wday(started_at, label = TRUE, abbr = TRUE)
  )

# Verificar primeros registros
head(select(df, ride_id, started_at, ended_at, duration_min, duration_min_calc, day_of_week, day_of_week_text))

In [ ]:
# Filtrar viajes con duraciones inválidas o atípicas
# Eliminar viajes con duración igual o menor a cero minutos
df <- df %>% 
  filter(duration_min > 0)

# Eliminar viajes con duración excesiva (más de 180 minutos = 3 horas)
df <- df %>% 
  filter(duration_min < 180)

# Verificar cantidad de registros tras la limpieza
df %>% summarise(n = n())


In [ ]:
# Crear variables adicionales para análisis temporal
library(lubridate)
library(dplyr)

# Verificar y convertir 'started_at' a formato datetime si fuera necesario
if (!inherits(df$started_at, "POSIXct")) {
  df$started_at <- ymd_hms(df$started_at)
}

# Enriquecer el dataset con variables temporales
df <- df %>%
  mutate(
    # Mes del viaje (abreviado)
    month = month(started_at, label = TRUE, abbr = TRUE),
    # Indicador de fin de semana
    is_weekend = if_else(day_of_week_text %in% c("Sat", "Sun"), "weekend", "weekday"),
    # Hora de inicio del viaje
    hour_of_day = hour(started_at),
    # Franja horaria categorizada
    part_of_day = case_when(
      hour_of_day >= 5 & hour_of_day < 12 ~ "morning",
      hour_of_day >= 12 & hour_of_day < 18 ~ "afternoon",
      hour_of_day >= 18 & hour_of_day < 24 ~ "evening",
      TRUE ~ "night"
    )
  )
# Vista previa de las nuevas variables
head(df[, c("started_at", "month", "day_of_week_text", "is_weekend", "hour_of_day", "part_of_day")])


---

## Análisis Exploratorio de Datos (EDA) - Versión preliminar

El objetivo de este análisis es comprender las diferencias en el comportamiento de viaje entre **usuarios casuales** y **miembros** del sistema de bicicletas Cyclistic, utilizando la versión preliminar del dataset (con datos incompletos de marzo 2024).  

Se exploraron distintas dimensiones del comportamiento de uso, mediante visualizaciones que permiten detectar patrones clave para futuras estrategias de marketing, producto y operaciones.

### Aspectos analizados:

1. **Duración del viaje**
   - Resumen estadístico de la duración por tipo de usuario
   - Duración promedio por franja horaria
   - Duración promedio según tipo de bicicleta

2. **Frecuencia de uso**
   - Número de viajes por día de la semana
   - Cantidad de viajes por mes
   - Cantidad de viajes por hora del día

3. **Preferencias y puntos de uso**
   - Uso de bicicletas según tipo (`classic`, `electric`, `docked`)
   - Top 10 estaciones de inicio más utilizadas

---

Estas visualizaciones permiten identificar **tendencias diferenciales** entre usuarios casuales y miembros, fundamentales para tomar decisiones basadas en datos.

In [ ]:
# Resumen estadístico duración por tipo de usuario
df %>%
  group_by(member_casual) %>%
  summarise(
    viajes = n(),
    duracion_promedio = mean(duration_min, na.rm = TRUE),
    duracion_mediana = median(duration_min, na.rm = TRUE)
  ) %>%
  arrange(desc(duracion_promedio))

Los usuarios casuales realizan viajes con una duración promedio y mediana significativamente mayor que los usuarios miembros. En esta versión preliminar, los casuales promedian 15,1 minutos por viaje (mediana de 9 min), mientras que los miembros promedian 10,1 minutos (mediana de 7 min).

Esto sugiere un comportamiento diferenciado: los miembros parecen usar el sistema para desplazamientos más cortos y posiblemente regulares (como traslados al trabajo o actividades diarias), mientras que los usuarios casuales tienden a realizar recorridos más largos, posiblemente con fines recreativos o turísticos.


In [ ]:
# Número de viajes por día de la semana y tipo de usuario 
library(ggplot2)

df %>%
  count(day_of_week_text, member_casual) %>%
  ggplot(aes(x = day_of_week_text, y = n, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(
    title = "Cantidad de viajes por día de la semana y tipo de usuario",
    x = "Día de la semana",
    y = "Cantidad de viajes",
    fill = "Tipo de usuario"
  ) +
  theme_minimal()

Los usuarios miembros mantienen un volumen alto de viajes durante toda la semana, especialmente de lunes a jueves, lo que sugiere un uso frecuente para desplazamientos laborales. Los usuarios casuales muestran mayor actividad los fines de semana, lo que puede reflejar un uso recreativo.

Esto refuerza la hipótesis de que ambos grupos utilizan el sistema con motivaciones distintas: rutina diaria vs. ocio ocasional.


In [ ]:
# Duración promedio por franja horaria y tipo de usuario 
df %>%
  group_by(part_of_day, member_casual) %>%
  summarise(duracion_promedio = mean(duration_min, na.rm = TRUE)) %>%
  ggplot(aes(x = part_of_day, y = duracion_promedio, color = member_casual, group = member_casual)) +
  geom_line(size = 1.2) +
  geom_point(size = 3) +
  labs(
    title = "Duración promedio de viaje por franja horaria y tipo de usuario",
    x = "Franja horaria",
    y = "Duración promedio (minutos)",
    color = "Tipo de usuario"
  ) +
  theme_minimal()

Los usuarios casuales realizan viajes más largos durante la tarde, mientras que los miembros mantienen una duración corta y estable en todas las franjas horarias.

La franja vespertina parece ser clave para la audiencia casual, lo que podría orientar promociones o contenido publicitario a esas horas.


In [ ]:
# Análisis de uso por tipo de bicicleta y usuario
df %>%
  count(rideable_type, member_casual) %>%
  ggplot(aes(x = rideable_type, y = n, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(
    title = "Cantidad de viajes según tipo de bicicleta y tipo de usuario",
    x = "Tipo de bicicleta",
    y = "Cantidad de viajes",
    fill = "Tipo de usuario"
  ) +
  theme_minimal()

Los miembros prefieren ampliamente las bicicletas clásicas, mientras que los casuales presentan una distribución más equilibrada entre bicicletas clásicas y eléctricas.

Esto podría indicar que los casuales eligen las eléctricas por su comodidad en trayectos turísticos, mientras que los miembros priorizan eficiencia y costo.

In [ ]:
# Análisis de duración promedio por tipo de bicicleta y usuario
df %>%
  group_by(rideable_type, member_casual) %>%
  summarise(duracion_promedio = mean(duration_min, na.rm = TRUE), .groups = "drop") %>%
  ggplot(aes(x = rideable_type, y = duracion_promedio, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(
    title = "Duración promedio de viajes según tipo de bicicleta y usuario",
    x = "Tipo de bicicleta",
    y = "Duración promedio (minutos)",
    fill = "Tipo de usuario"
  ) +
  theme_minimal()

Los usuarios casuales presentan mayor duración promedio en ambos tipos de bicicletas, especialmente en bicicletas clásicas. Los miembros, en cambio, mantienen una duración más baja y homogénea, independientemente del tipo.

Esto refuerza la idea de un uso más recreativo por parte de los casuales, que estarían interesados en recorridos largos, frente a un uso funcional por parte de los miembros.

In [ ]:
# Análisis de viajes por estación de inicio (top 10 estaciones)
df %>%
  count(start_station_name) %>%
  arrange(desc(n)) %>%
  slice_head(n = 10) %>%
  ggplot(aes(x = reorder(start_station_name, n), y = n)) +
  geom_col(fill = "#1f77b4") +
  coord_flip() +
  labs(
    title = "Top 10 estaciones de inicio con mayor cantidad de viajes",
    x = "Estación de inicio",
    y = "Cantidad de viajes"
  ) +
  theme_minimal()

Las estaciones más utilizadas se concentran en zonas céntricas y de alta actividad, como Clinton St, University Ave y Kingsbury St. Esto puede deberse a la cercanía con oficinas, universidades o zonas turísticas.

Identificar estas estaciones clave puede ayudar a Cyclistic a enfocar campañas de marketing o a reforzar infraestructura donde hay mayor demanda.


In [ ]:
# Análisis de cantidad de viajes por hora del día y tipo de usuario
df %>%
  filter(!is.na(hour_of_day)) %>%
  count(hour_of_day, member_casual) %>%
  ggplot(aes(x = hour_of_day, y = n, color = member_casual)) +
  geom_line(size = 1.2) +
  geom_point(size = 2) +
  labs(
    title = "Cantidad de viajes por hora del día y tipo de usuario",
    x = "Hora del día",
    y = "Cantidad de viajes",
    color = "Tipo de usuario"
  ) +
  theme_minimal()

Los miembros muestran dos picos bien marcados de uso entre las 7-9 h y 17-18 h, consistentes con horarios laborales. Los casuales, en cambio, presentan un uso más distribuido a lo largo del día, con un pico leve entre las 14-17 h.

Esto sugiere que los miembros utilizan las bicicletas para traslados diarios, mientras que los casuales aprovechan horarios de mayor disponibilidad, posiblemente por ocio o turismo.

In [ ]:
# Análisis de cantidad de viajes por mes y tipo de usuario
df %>%
  count(month, member_casual) %>%
  ggplot(aes(x = month, y = n, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(
    title = "Cantidad de viajes por mes y tipo de usuario",
    x = "Mes",
    y = "Cantidad de viajes",
    fill = "Tipo de usuario"
  ) +
  theme_minimal()

**(versión preliminar)** Febrero aparece como el mes con mayor cantidad de viajes tanto para usuarios miembros como casuales. Marzo muestra muy pocos registros, lo que indica una posible inconsistencia en la base de datos.

⚠️ *Este análisis se realizó antes de detectar que el dataset de marzo estaba incompleto. El resultado será reevaluado con los datos corregidos.*


---

## Revisión crítica del proceso de limpieza

Durante la validación inicial del análisis se detectó una anomalía significativa en la distribución de registros por mes: el mes de marzo 2024 contenía únicamente **78 registros**, en contraste con más de 100.000 registros en enero y febrero.

Al revisar el flujo de trabajo, se identificó que la causa fue una limpieza excesiva realizada de forma manual en Google Sheets, donde se eliminaron filas con valores nulos sin un control riguroso sobre la integridad y representatividad de los datos. Esto provocó una reducción drástica en el volumen de registros válidos para marzo.

Este hallazgo motivó una revisión integral del enfoque de limpieza, reemplazando el procedimiento inicial por uno más robusto y controlado que incluyó:

- Reimportar directamente los archivos originales de enero, febrero y marzo en BigQuery.
- Aplicar procesos de validación y depuración mediante consultas SQL.
- Unificar los datasets mensuales en una tabla consolidada.

Como resultado, se obtuvo una base de datos completa con **652.834 registros válidos** correspondientes al primer trimestre de 2024.

---

⚠️ Este tipo de situaciones es común en proyectos reales de análisis de datos. La capacidad para detectar inconsistencias, rastrear su origen y ajustar el proceso demuestra un enfoque analítico riguroso y profesional.


> 🔍 **Observación personal:**  
> Este fue el punto de inflexión del proyecto. Detectar el sesgo me permitió validar la importancia de revisar volúmenes en etapas tempranas, y fortaleció mi mirada crítica sobre los datos antes de continuar el análisis.

---

## Limpieza de datos realizada en BigQuery

Tras identificar inconsistencias derivadas del proceso manual previo en Google Sheets, se procedió a reimportar directamente en BigQuery los archivos CSV originales correspondientes a los meses de **enero, febrero y marzo de 2024**.

A partir de esta base confiable, se aplicó un proceso de limpieza completo y uniforme que contempló:

- Eliminación de registros con valores nulos en columnas clave: `start_station_id`, `end_station_id`, `start_lat`, `start_lng`, `end_lat` y `end_lng`.
- Conversión de las columnas de fecha y hora al tipo de dato `TIMESTAMP`.
- Cálculo de la duración de cada viaje en minutos (`duration_min`).
- Exclusión de registros con duración superior a 300 minutos, considerados viajes atípicos o erróneos.
- Creación de nuevas variables para enriquecer el análisis, tales como:
  - Día de la semana (`day_of_week`)
  - Nombre del día de la semana en texto (`day_of_week_text`)
  - Hora de inicio del viaje (`start_hour`)
  - Franja horaria o turno (`time_slot`)

Este enfoque, implementado y validado paso a paso mediante consultas SQL, permitió obtener una tabla final consolidada, limpia y confiable:  
**`tripsdata_q1_2024`, con 652.834 registros válidos.**

> 💬 **Nota del analista:**  
> A fin de evitar una pérdida excesiva de registros, se decidió conservar valores nulos en columnas no críticas como `start_station_name` y `end_station_name`. Estas columnas presentan un **12,68%** y **13,25%** de datos faltantes respectivamente. Esta decisión preservó la representatividad del dataset completo, aunque introduce la categoría “NA” en ciertos gráficos, como el de estaciones de inicio, donde se refleja este fenómeno como una agrupación de viajes sin georreferenciación identificada.



---

> 🛠️ **Elección de herramienta:**  
> Opté por trabajar en BigQuery para la limpieza y consolidación porque ofrecía mayor estabilidad y escalabilidad al unir múltiples archivos. Esta decisión mejoró la integridad del dataset y facilitó la preparación de la tabla final.

In [ ]:
# Cargar el nuevo dataset limpio exportado desde BigQuery

# Ruta al archivo importado en Kaggle 
ruta_csv <- "/kaggle/input/tripsdata-q1-2024-corregido/tripsdata_q1_2024.csv"

# Leer el archivo CSV con encabezado
df <- readr::read_csv(ruta_csv, show_col_types = FALSE)

# Limpiar nombres de columnas 
df <- janitor::clean_names(df)

# Revisar las primeras filas y estructura general
head(df)
glimpse(df)

# Verificar cantidad de registros y columnas
dim(df)


---

## Validación de la importación del dataset final

Luego de completar la limpieza y consolidación de los tres archivos originales correspondientes a enero, febrero y marzo de 2024, se exportó la tabla `tripsdata_q1_2024` desde BigQuery como archivo CSV y se importó en este entorno de Kaggle.

Para validar que la importación fue exitosa, se revisó la estructura del dataframe, obteniendo el siguiente resultado:

- **Registros:** 652.834
- **Columnas:** 16
- **Variables principales:**
  - `ride_id`: Identificador único de cada viaje
  - `rideable_type`: Tipo de bicicleta utilizada
  - `started_at` / `ended_at`: Fecha y hora de inicio y fin
  - `start_lat` / `start_lng` / `end_lat` / `end_lng`: Coordenadas de salida y llegada
  - `member_casual`: Tipo de usuario
  - `duration_min`: Duración del viaje en minutos
  - `day_of_week` / `day_of_week_text`: Día de la semana en formato numérico y textual

La cantidad de registros y variables coincide con lo esperado, y los nombres de columnas se importaron correctamente. Con esta validación completada, podemos continuar con el análisis sobre datos confiables y actualizados.


In [ ]:
# Conversión de columnas de fecha y hora a formato datetime

# Convertir las columnas started_at y ended_at al tipo POSIXct
df$started_at <- as.POSIXct(df$started_at, format = "%Y-%m-%d %H:%M:%S", tz = "UTC")
df$ended_at <- as.POSIXct(df$ended_at, format = "%Y-%m-%d %H:%M:%S", tz = "UTC")

# Validar que la conversión fue exitosa
glimpse(df)


In [ ]:
# Crear variables adicionales para análisis temporal

df <- df %>%
  mutate(
    # Hora de inicio del viaje
    hour_of_day = hour(started_at),
    # Franja horaria categorizada
    part_of_day = case_when(
      hour_of_day >= 0 & hour_of_day < 6 ~ "00-06",
      hour_of_day >= 6 & hour_of_day < 12 ~ "06-12",
      hour_of_day >= 12 & hour_of_day < 18 ~ "12-18",
      TRUE ~ "18-00"
    ),
    part_of_day = factor(part_of_day, levels = c("00-06", "06-12", "12-18", "18-00"), ordered = TRUE)
  )


In [ ]:
# Crear columna para mes del viaje
df <- df %>%
  mutate(
    month = month(started_at),
    month = case_when(
      month == 1 ~ "enero",
      month == 2 ~ "febrero",
      month == 3 ~ "marzo"
    ),
    month = factor(month, levels = c("enero", "febrero", "marzo"), ordered = TRUE)
  )

---

## Nuevo Análisis Exploratorio de Datos (EDA)

Tras detectar un error en la tabla original, se realizó una limpieza y unificación exhaustiva de las bases de datos. A partir de esta tabla corregida, retomamos el análisis exploratorio para validar y actualizar los hallazgos sobre el comportamiento de los usuarios casuales y miembros.

Este análisis renovado permitirá obtener insights más confiables sobre:

- La proporcion de viajes por tipo de usuario.
- El patrón de viajes por día de la semana y tipo de usuario.
- La duración promedio en diferentes franjas horarias.
- El uso y duración de viajes según el tipo de bicicleta.
- Las estaciones con mayor actividad.
- La distribución de viajes por hora del día y mes, segmentada por tipo de usuario.

A continuación, se presentan los gráficos y análisis actualizados.


In [ ]:
# Resumen estadístico duración por tipo de usuario
df %>%
  group_by(member_casual) %>%
  summarise(
    viajes = n(),
    duracion_promedio = mean(duration_min, na.rm = TRUE),
    duracion_mediana = median(duration_min, na.rm = TRUE)
  ) %>%
  arrange(desc(duracion_promedio))

In [ ]:
# Proporción de usuarios por tipo (miembros vs casuales)
df %>%
  count(member_casual) %>%
  mutate(porcentaje = n / sum(n) * 100,
         etiqueta = paste0(member_casual, " (", round(porcentaje, 1), "%)")) %>%
  ggplot(aes(x = "", y = porcentaje, fill = member_casual)) +
  geom_col(width = 1) +
  coord_polar(theta = "y") +
  labs(
    title = "Distribución porcentual por tipo de usuario",
    fill = "Tipo de usuario"
  ) +
  theme_void() +
  geom_text(aes(label = etiqueta), position = position_stack(vjust = 0.5))


La mayoría de los viajes (77,1%) son realizados por usuarios miembros, lo que indica que el sistema de suscripción representa el principal canal de uso de bicicletas compartidas.
Por el contrario, los usuarios casuales representan solo el 22,9%, lo que señala una oportunidad de crecimiento si se desarrollan estrategias efectivas de conversión.
Esta proporción también justifica el enfoque del negocio en retener y expandir la base de miembros, sin dejar de lado el potencial latente en los usuarios casuales.

In [ ]:
# Número de viajes por día de la semana y tipo de usuario 
library(ggplot2)

df %>%
  count(day_of_week_text, member_casual) %>%
  ggplot(aes(x = day_of_week_text, y = n, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(
    title = "Cantidad de viajes por día de la semana y tipo de usuario",
    x = "Día de la semana",
    y = "Cantidad de viajes",
    fill = "Tipo de usuario"
  ) +
  theme_minimal()

Usuarios miembros tienen un uso mucho más elevado y constante durante toda la semana, destacándose los días laborables (especialmente jueves y martes).
Usuarios casuales muestran mayor actividad relativa los fines de semana, especialmente sábados, lo que sugiere un uso más recreativo o turístico.

In [ ]:
# Duración promedio por franja horaria y tipo de usuario (versión sin advertencias)
df %>%
  group_by(part_of_day, member_casual) %>%
  summarise(duracion_promedio = mean(duration_min, na.rm = TRUE), .groups = "drop") %>%
  ggplot(aes(x = part_of_day, y = duracion_promedio, color = member_casual, group = member_casual)) +
  geom_line(linewidth = 1.2) +
  geom_point(size = 3) +
  labs(
    title = "Duración promedio de viaje por franja horaria y tipo de usuario",
    x = "Franja horaria",
    y = "Duración promedio (minutos)",
    color = "Tipo de usuario"
  ) +
  theme_minimal()

Los usuarios casuales realizan viajes más largos en todas las franjas horarias, con un pico entre las 12h y 18h (18 minutos).
En contraste, los miembros muestran duraciones estables (10-11 minutos), lo que refuerza la hipótesis de un uso más funcional y cotidiano.

In [ ]:
# Análisis de uso por tipo de bicicleta y usuario
df %>%
  count(rideable_type, member_casual) %>%
  ggplot(aes(x = rideable_type, y = n, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(
    title = "Cantidad de viajes según tipo de bicicleta y tipo de usuario",
    x = "Tipo de bicicleta",
    y = "Cantidad de viajes",
    fill = "Tipo de usuario"
  ) +
  theme_minimal()

Tanto miembros como casuales utilizan mayoritariamente bicicletas clásicas, pero los miembros duplican o triplican el volumen en ambas categorías.
Esto indica un uso más intensivo por parte de los suscriptores, sin una preferencia excluyente por el tipo de bicicleta.

In [ ]:
# Análisis de duración promedio por tipo de bicicleta y usuario
df %>%
  group_by(rideable_type, member_casual) %>%
  summarise(duracion_promedio = mean(duration_min, na.rm = TRUE), .groups = "drop") %>%
  ggplot(aes(x = rideable_type, y = duracion_promedio, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(
    title = "Duración promedio de viajes según tipo de bicicleta y usuario",
    x = "Tipo de bicicleta",
    y = "Duración promedio (minutos)",
    fill = "Tipo de usuario"
  ) +
  theme_minimal()

Los usuarios casuales tienen viajes sustancialmente más largos, especialmente con bicicletas clásicas (21 minutos).
Esto refuerza el uso recreativo y posiblemente más esporádico, comparado con el uso más corto y eficiente de los miembros (10 minutos).

In [ ]:
# Análisis de viajes por estación de inicio (top 10 estaciones)
df %>%
  count(start_station_name) %>%
  arrange(desc(n)) %>%
  slice_head(n = 10) %>%
  ggplot(aes(x = reorder(start_station_name, n), y = n)) +
  geom_col(fill = "#1f77b4") +
  coord_flip() +
  labs(
    title = "Top 10 estaciones de inicio con mayor cantidad de viajes",
    x = "Estación de inicio",
    y = "Cantidad de viajes"
  ) +
  theme_minimal()

Las estaciones con mayor cantidad de viajes están dominadas por **ubicaciones céntricas y turísticas**, lo cual es consistente con el uso ocasional de los usuarios casuales y la densidad urbana. 

> Sin embargo, el valor “NA” aparece como la categoría más frecuente: esto **no representa una estación real**, sino un conjunto de viajes con el campo `start_station_name` vacío (12,68% del total).  
> Esta limitación en la calidad del dato fue documentada y considerada al interpretar los resultados, sin afectar el resto del análisis.


In [ ]:
# Análisis de cantidad de viajes por hora del día y tipo de usuario
df %>%
  filter(!is.na(hour_of_day)) %>%
  count(hour_of_day, member_casual) %>%
  ggplot(aes(x = hour_of_day, y = n, color = member_casual)) +
  geom_line(size = 1.2) +
  geom_point(size = 2) +
  labs(
    title = "Cantidad de viajes por hora del día y tipo de usuario",
    x = "Hora del día",
    y = "Cantidad de viajes",
    color = "Tipo de usuario"
  ) +
  theme_minimal()

Miembros presentan dos picos marcados: 7–9 h y 17–18 h, coincidentes con horarios laborales.
Casuales tienen una curva más uniforme con mayor actividad entre las 11 y 17 h, típica de actividades recreativas o de ocio.

In [ ]:
# Análisis de cantidad de viajes por mes y tipo de usuario
df %>%
  count(month, member_casual) %>%
  ggplot(aes(x = month, y = n, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(
    title = "Cantidad de viajes por mes y tipo de usuario",
    x = "Mes",
    y = "Cantidad de viajes",
    fill = "Tipo de usuario"
  ) +
  theme_minimal()


El uso crece mes a mes tanto para miembros como casuales, siendo marzo el mes con más viajes.
Esto puede estar asociado a la estacionalidad (mejor clima), pero también sugiere potencial de mayor captación de usuarios en primavera/verano.

---

## Revisión comparativa entre análisis preliminar y análisis con datos íntegros

Con el objetivo de garantizar la validez de los hallazgos y recomendaciones, esta sección presenta una comparación entre los resultados obtenidos con la **tabla preliminar sesgada** (primer análisis exploratorio) y los resultados finales generados con la **tabla íntegra corregida**, tras detectar y resolver problemas de subregistro.

---

### ¿Qué vas a encontrar a continuación?

En los próximos bloques se describen y visualizan las diferencias más relevantes entre ambos enfoques. Se busca mostrar cómo ciertos hallazgos iniciales se vieron alterados de forma significativa al trabajar con una base de datos completa y confiable.

Cada punto incluye:
- Una breve descripción del hallazgo en el análisis preliminar  
- El contraste con el resultado final corregido 
- Los gráficos correspondientes para facilitar la visualización comparativa  



---
### Comparación 1: Evolución mensual del uso (tabla sesgada vs. íntegra)

> *El dato faltante de marzo en la tabla sesgada generaba una falsa señal de estancamiento en la demanda.*

Con los datos corregidos, marzo resultó ser el mes con mayor actividad, evidenciando una **tendencia de crecimiento estacional** clave para la toma de decisiones comerciales.

El siguiente gráfico muestra la diferencia entre ambos enfoques:

- **Izquierda:** tabla sesgada sin registros de marzo.
- **Derecha:** tabla íntegra con comportamiento real de usuarios.

In [ ]:
# Cantidad de viajes por mes
#Cargar librerías
library(tidyverse)
library(lubridate)
library(patchwork)  
# Para unir gráficos lado a lado

# ─────────────────────────────────────────────
# Cargar y preparar datasets

# Dataset sesgado
data_sesgada <- read_csv("/kaggle/input/cyclistic-casestudy/tripsdata_q1_2024.csv", col_names = FALSE, show_col_types = FALSE)
colnames(data_sesgada) <- c(
  "ride_id", "rideable_type", "started_at", "ended_at", "start_station_name",
  "start_station_id", "end_station_name", "end_station_id", "start_lat", "start_lng",
  "end_lat", "end_lng", "member_casual", "ride_length", "day_of_week", "duration_min"
)
data_sesgada <- data_sesgada %>%
  mutate(started_at = ymd_hms(started_at))

# Dataset íntegro
data_integra <- read_csv("/kaggle/input/tripsdata-q1-2024-corregido/tripsdata_q1_2024.csv", show_col_types = FALSE)
data_integra <- data_integra %>%
  mutate(started_at = ymd_hms(started_at))

# ─────────────────────────────────────────────
# Procesamiento

prep_viajes_mensuales <- function(df) {
  df %>%
    mutate(mes = month(started_at, label = TRUE, abbr = TRUE)) %>%
    group_by(mes, member_casual) %>%
    summarise(viajes = n(), .groups = "drop")
}

viajes_sesgada <- prep_viajes_mensuales(data_sesgada)
viajes_integra <- prep_viajes_mensuales(data_integra)

# ─────────────────────────────────────────────
# Crear gráficos comparativos

graf_sesgada <- ggplot(viajes_sesgada, aes(x = mes, y = viajes, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(title = "Tabla Sesgada", x = "Mes", y = "Cantidad de viajes", fill = "Tipo de usuario") +
  theme_minimal()

graf_integra <- ggplot(viajes_integra, aes(x = mes, y = viajes, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(title = "Tabla Íntegra", x = "Mes", y = "Cantidad de viajes", fill = "Tipo de usuario") +
  theme_minimal()

# Mostrar ambos gráficos lado a lado
graf_sesgada + graf_integra


---
### Comparación 2: Duración promedio y mediana por tipo de usuario

> *¿Cuánto tiempo pedalea cada tipo de usuario y cómo cambia esa métrica con datos corregidos?*

Ambos análisis muestran que los **usuarios casuales tienen trayectos más largos** que los miembros, tanto en promedio como en mediana. Sin embargo, con la tabla íntegra el cálculo es más estable y representativo.

- El promedio de los casuales bajó levemente al incluir más datos.
- Las medianas se mantuvieron estables, reforzando el patrón de mayor duración recreativa en este segmento.

*Conclusión:* Aunque los patrones se conservaron, la tabla íntegra mejora la precisión de las métricas, especialmente al reducir el riesgo de sobreestimaciones por valores atípicos.


In [ ]:
# Resumen de duración promedio y mediana

prep_duracion_resumen <- function(df) {
  df %>%
    group_by(member_casual) %>%
    summarise(
      promedio = mean(duration_min, na.rm = TRUE),
      mediana  = median(duration_min, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    pivot_longer(cols = c(promedio, mediana), names_to = "tipo", values_to = "duracion")
}

duracion_sesgada <- prep_duracion_resumen(data_sesgada)
duracion_integra <- prep_duracion_resumen(data_integra)

# ─────────────────────────────────────────────
# Crear gráficos

graf_duracion_sesgada <- ggplot(duracion_sesgada, aes(x = member_casual, y = duracion, fill = tipo)) +
  geom_col(position = "dodge") +
  labs(title = "Tabla Sesgada", x = "Tipo de usuario", y = "Duración (minutos)", fill = "Métrica") +
  theme_minimal()

graf_duracion_integra <- ggplot(duracion_integra, aes(x = member_casual, y = duracion, fill = tipo)) +
  geom_col(position = "dodge") +
  labs(title = "Tabla Íntegra", x = "Tipo de usuario", y = "Duración (minutos)", fill = "Métrica") +
  theme_minimal()

# Unir ambos gráficos
graf_duracion_sesgada + graf_duracion_integra

---
### Comparación 3: Distribución horaria de viajes por tipo de usuario

> *¿En qué momentos del día se utiliza más el sistema y cómo afecta la cantidad total de registros esa lectura?*

Ambos análisis muestran una pauta clara:
- Los **miembros** concentran su actividad en horarios pico (7-9 h y 17-18 h), lo que sugiere un uso vinculado al trabajo o estudio.
- Los **casuales** muestran una distribución más pareja, con foco entre las 11 y 18 h, consistente con fines recreativos o turísticos.

Con la tabla íntegra, los volúmenes se duplican y permiten una lectura más robusta. Aunque el patrón general se conserva, el gráfico sesgado **subestima el impacto real en horas clave**, lo que podría afectar decisiones operativas (disponibilidad, mantenimiento, marketing).

*Conclusión:* Validar el volumen real es clave para dimensionar correctamente la demanda y ajustar recursos en horarios críticos.

In [ ]:
# Cantidad de viajes por hora del día por tipo de usuario
# ─────────────────────────────────────────────
# Procesamiento: agrupar por hora del día

prep_viajes_por_hora <- function(df) {
  df %>%
    mutate(hora = hour(started_at)) %>%
    group_by(hora, member_casual) %>%
    summarise(viajes = n(), .groups = "drop")
}

viajes_hora_sesgada <- prep_viajes_por_hora(data_sesgada)
viajes_hora_integra <- prep_viajes_por_hora(data_integra)

# ─────────────────────────────────────────────
# Crear gráficos

graf_hora_sesgada <- ggplot(viajes_hora_sesgada, aes(x = hora, y = viajes, color = member_casual)) +
  geom_line(size = 1) +
  geom_point() +
  labs(title = "Tabla Sesgada", x = "Hora del día", y = "Cantidad de viajes", color = "Tipo de usuario") +
  theme_minimal()

graf_hora_integra <- ggplot(viajes_hora_integra, aes(x = hora, y = viajes, color = member_casual)) +
  geom_line(size = 1) +
  geom_point() +
  labs(title = "Tabla Íntegra", x = "Hora del día", y = "Cantidad de viajes", color = "Tipo de usuario") +
  theme_minimal()

# Mostrar lado a lado
graf_hora_sesgada + graf_hora_integra


---

### Comparación 4: Top 10 estaciones de inicio

> *¿Dónde comienza la mayoría de los viajes y cómo influye la calidad del dato en esta lectura geográfica?*

Ambos análisis coinciden en que ciertas estaciones céntricas (como *Clinton St & Washington Blvd* o *University Ave & 57th St*) concentran gran parte del uso del sistema.

Sin embargo, con el dataset íntegro:
- Aumenta significativamente el volumen de viajes en cada estación.
- Se suman nuevas estaciones al ranking (ej. *Streeter Dr & Grand Ave*), que habían quedado fuera por falta de datos.

*Conclusión:* La tabla sesgada subestima puntos de alta demanda. Esto podría llevar a decisiones equivocadas en logística, mantenimiento o marketing geolocalizado.


In [ ]:
# Top 10 estaciones de inicio por cantidad de viajes
# Procesamiento con seguridad
prep_top_stations <- function(df) {
  df %>%
    filter(!is.na(start_station_name)) %>%
    group_by(start_station_name) %>%
    summarise(viajes = n(), .groups = "drop") %>%
    arrange(desc(viajes)) %>%
    slice_head(n = 10)
}

# Generar top 10
top_stations_sesgada <- prep_top_stations(data_sesgada)
top_stations_integra <- prep_top_stations(data_integra)

# Gráficos individuales
graf_estaciones_sesgada <- ggplot(top_stations_sesgada, aes(x = reorder(start_station_name, viajes), y = viajes)) +
  geom_col(fill = "#0099cc") +
  coord_flip() +
  labs(title = "Tabla Sesgada", x = "Estación de inicio", y = "Cantidad de viajes") +
  theme_minimal()

graf_estaciones_integra <- ggplot(top_stations_integra, aes(x = reorder(start_station_name, viajes), y = viajes)) +
  geom_col(fill = "#00bfff") +
  coord_flip() +
  labs(title = "Tabla Íntegra", x = "Estación de inicio", y = "Cantidad de viajes") +
  theme_minimal()

# Mostrar con print() explícito
print(graf_estaciones_sesgada + graf_estaciones_integra)

---

### Hallazgos clave del análisis (tabla íntegra)

> Resumen de los principales comportamientos detectados tras validar los datos y eliminar el sesgo inicial:

- Los **usuarios miembros** representan el 77% de los viajes y tienden a:
  - Usar bicicletas tradicionales
  - Viajar en días hábiles, con picos en horarios laborales (8 y 17 hs)
  - Realizar trayectos más breves pero frecuentes
  - Tener un perfil funcional, ligado a la rutina
    

- Los **usuarios casuales** realizan menos viajes, pero:
  - Usan bicicletas eléctricas con mayor frecuencia
  - Viajan principalmente por la tarde y los fines de semana
  - Permanecen más tiempo en cada trayecto
  - Tienen un perfil recreativo u ocasional

- **Estaciones céntricas compartidas** como *Kingsbury & Kinzie* o *Ellis & 60th* concentran usuarios de ambos perfiles: puntos clave de conversión.


---

### Recomendaciones y próximos pasos

> Con base en el análisis de comportamiento, se proponen las siguientes acciones:

#### Recomendaciones

- **Segmentar campañas por perfil de uso**
  - Enfocar mensajes funcionales a miembros potenciales
  - Promover beneficios para uso diario (ahorro, rapidez, disponibilidad)

- **Ofrecer pruebas de membresía**
  - Descuentos o membresías temporales para usuarios casuales frecuentes

- **Acciones en estaciones clave**
  - Intervenciones directas en puntos con alta concurrencia mixta

- **Promover el uso funcional de la bicicleta eléctrica**
  - Visibilizar ventajas para movilidad urbana, más allá del ocio

#### Próximos pasos

- Incluir **motivo de uso** mediante encuestas o inputs voluntarios
- Incorporar variables demográficas (edad, género, barrio)
- Cruzar datos con eventos urbanos, clima o turismo
- Ampliar el análisis a todo 2024 para validar estacionalidad
- Testear campañas piloto en estaciones con alto potencial de conversión


---

### Conclusión del proceso: del dato a la experiencia

Este caso marcó mi primera experiencia recorriendo un proyecto completo de análisis de datos, no solo como actividad aislada, sino como un flujo real que integra limpieza, validación, visualización y comunicación.

#### Error inicial, aprendizaje duradero

La omisión de registros en marzo me llevó a construir presentaciones visuales (Tableau + Canva) sobre una base sesgada. Este error fue clave: me mostró que la validación rigurosa de los datos debe hacerse antes de cualquier comunicación externa.

#### Herramientas: lo que volvería a hacer (y lo que no)

Hoy, con una mirada más entrenada, elegiría:
- **BigQuery** para limpieza, transformación y fusión robusta.
- **Kaggle** para exploración y visualización inicial (ideal para detectar errores).
- **Tableau** para dashboards, y **Canva** para presentaciones a públicos no técnicos.

Haber intentado definir herramientas “sobre la marcha” fue parte de mi curva de aprendizaje. Me sirvió para descubrir el valor real de cada plataforma, pero también para entender que una **buena planificación inicial** evita retrabajos y desalineaciones.

#### Evolución profesional

Al comienzo, me faltaba confianza para tomar decisiones técnicas. Ahora entiendo que elegir bien las herramientas desde el inicio y validar cada paso es lo que convierte un ejercicio técnico en un análisis profesional.

> Esta experiencia no solo mejoró mi dominio técnico; también fortaleció mi criterio analítico y mi capacidad para estructurar un proceso de principio a fin.



---

## Si solo vas a leer una sección, que sea esta

Este proyecto analiza el comportamiento de usuarios de Cyclistic, un sistema de bicicletas compartidas en Chicago, con el objetivo de **convertir usuarios casuales en miembros anuales**.

 A lo largo del análisis:

- Se detectó un **sesgo crítico en los datos preliminares**, que omitía un mes completo (marzo).  
- Se reconstruyó el dataset completo usando BigQuery y se validaron todas las métricas.  
- Se desarrollaron visualizaciones comparativas en Tableau y análisis reproducibles en Kaggle.

 Hallazgos clave:

- **Miembros**: uso funcional, días hábiles, bicicletas tradicionales, trayectos breves y frecuentes.  
- **Casuales**: uso recreativo, fines de semana, bicicletas eléctricas, trayectos más largos.

 Recomendaciones:

- Segmentar campañas por perfil de uso.  
- Ofrecer membresías temporales y beneficios visibles para casuales frecuentes.  
- Intervenir estaciones compartidas con acciones de conversión.

 Conclusión:

> Este caso no solo entregó insights accionables, sino que fortaleció mi criterio técnico para validar datos, elegir herramientas y estructurar un análisis profesional de principio a fin.


---

## Autoría

Este análisis fue desarrollado por **Laura Agostina Spena**, Analista de Datos Junior, como parte del caso final para la **Certificación de Google Data Analytics** (Coursera, 2025).

El proyecto refleja una experiencia completa de trabajo con datos reales, incluyendo limpieza, transformación, análisis exploratorio, visualización, validación y comunicación de insights accionables.

📬 Para consultas profesionales o colaboración:

> **LinkedIn**: [linkedin.com/in/laura-agostina-spena-973652363](https://www.linkedin.com/in/laura-agostina-spena-973652363)  
> **Email**: spenalauraagostina@gmail.com  
